## Entendiendo el gradiente descendente

Los algoritmos de optimización por gradiente descendente son cada vez más populares, y a menudo son utilizados como cajas negras con explicaciones prácticas de sus fortalezas y sus debilidades. El objetivo de este artículo es proporcionarle al lector una pequeña intuición y los conocimientos mínimos en matemáticas para entender mejor el comportamiento de estos algoritmos y tener criterios sólidos para ponerlos en uso. En el curso de esta descripción general, estudiaremos las diferentes variantes de los gradientes descendentes y sus principales desafios. 

## Preliminares

El aprendizaje mecánico y la estadística comparten importantes características, pero usan una terminología diferente. En estadística, la regresión lineal se utiliza para modelar una relación $\mathcal{R}\subseteq\mathcal{X}\times\mathcal{Y}$ a partir  de una muestra de datos $S\subset \mathcal{R}\subseteq\mathcal{X}\times\mathcal{Y}\subseteq\mathbb{R}^{m+1}\times\mathbb{R}$. Usualmente los datos que provienen de $\mathcal{X}$ se denominan <em>variables independientes</em> y los que provienen de $\mathcal{Y}$ se denominan <em>variables dependientes</em>. En el contexto del aprendizaje mecánico, este es llamado un <em>problema de aprendizaje supervisado</em>. El conjunto $S=\{(x_i, y_i)\}_{i=0}^{k}$ se denomina <em>conjunto de entrenamiento</em> y el par de valores $(x_i, y_i)$ es un <em>ejemplo de entrenamiento</em> del conjunto $S$ de $k$ ejemplos de entrenamiento.

Cada $x_i$ es considerado como un vector columna de la forma $x_i=(1, x_{i,1},\dots,x_{i,m})^{\top}$ de dimensión $m+1$, a menudo es referido como el predictor en la literatura estadística. El proposito de la regresión lineal es identificar el mejor <em>predictor</em> de la clase de predictores de la forma $y =\theta^{\top} x$, donde $\theta \in \mathbb{R}^{m+1}$ y es de la forma $\theta =(\theta_0, \dots, \theta_m)^\top$, el parámetro $\theta_0$ se denomina <em>bias</em>. Para hacer esto, se considera la matriz $X\in M_{k\times m+1}(\mathbb{R})$ donde la $i$ - ésima fila está dada por el $i$ - ésimo vector fila $x_i^{\top}$. De esta manera se obtiene una expresión general para predecir los valores de $\mathcal{Y}$ que viene dada por el modelo:  

\begin{equation}\nonumber
Y=X\theta.
\end{equation} 

En Python podemos definir la clase de predictores lineales $y =\theta^{\top} x$ de la siguiente forma:

In [1]:
import numpy as np

class LinearPredictor:
    
    def __init__(self, m):
        """
        Input:
            theta: initial predictor.
            
        Output:
            None.
        """
        
        # The coefficient column vector is initialized to zero.
        self.theta = np.zeros(m + 1).reshape(-1, 1)
        
    def predict(self, x):
        """
        Input:
            x: list of float values of the form [1, x_1, ... ,x_m].
            
        Output:
            Returns the dot product (float) between «theta» and «x». 
        """
        
        # We define theta as self.theta.
        theta = self.theta        
        # Convert «x list» to a column vector.
        x = np.array(x).reshape(-1, 1)
        
        # Returns the product of the matrices theta and x.
        return float(theta.T.dot(x))

Observe que en el código anterior, el vector de coeficientes $\theta$ se puede inicializar a elección del lector, lo importante aquí, es ir actualizandolo hasta encontrar el mejor de ellos. Este problema de encontrar el mejor predictor lineal se resuelve eligiendo algún vector de coeficientes $\theta$ que minimizan la suma residual de cuadrados:

\begin{equation}\nonumber
E_S(\theta)=(Y-X\theta)^\top (Y-X\theta).
\end{equation}

En la ecuación anterior, $Y$ es el vector columna  donde la $i$ - ésima entrada está dada por cada la etiqueta $y_i$  definida en el conjunto de entrenamiento $S$, es decir, $Y = (y_1,\dots, y_k)^\top$. Esto permite definir el problema como un problema de minimización del riesgo empirico (ERM) dado por:

<a id='1'></a>
\begin{equation}\tag{1}
ERM_{\theta}(S)\equiv \operatorname*{argmin\,\,}_{ \theta \in \mathbb{R}^{m+1}} E_{S}(\theta),
\end{equation}

que es equivalente a  escoger un vector $\hat{\theta}\in ERM_{\theta}(S)$ para la solución del problema de optimización en la [Ecuación 1](#1). Para estudios de simulación, se puede construir un modelo sintético a partir de la generación de datos con: 

<a id='2'></a>
\begin{equation}\tag{2}
Y=X\xi + \epsilon,
\end{equation} 

donde $\epsilon\sim \mathcal{N}(0, \sigma^2I_{k})$. Esto se pueden implementar fácilmente en Python de la siguiente forma:

In [2]:
class SyntheticData:

    def __init__(self, m):
        """
        Input:
            m: dimension of the space of independent variables.
            
        Output:
            None.
        """
        
        # Dimensión of the data.
        self.m = m
        
        # The vector of coefficients is chosen at random.
        self.xi = np.random.rand(m + 1).reshape(-1, 1)        
          
    def generate(self, k, sigma):
        """
        Input:
            k: the amount of data to generate.
            sigma: standard deviation.
            
        Output:
            Returns a tuple with X, y and xi.
        """
        
        # A column vector of ones is constructed.
        ones = np.ones(k).reshape(-1, 1)
        
        # Random data is generated and added to the ones vector.
        X = np.hstack((ones, np.random.rand(k, self.m)))
        
        # Random noise is generated.
        epsilon = sigma * np.random.standard_normal(k)
        
        # Values for y are constructed.
        y = X.dot(self.xi) + epsilon.reshape(-1, 1)

        return X, y, self.xi

Para entender el comportamiendo del gradiente de descente es conveniente probar con diferentes colecciones de datos o lotes, para eso sea:

\begin{equation}\nonumber
    S_{l}=\{X_{l}, Y_{l}\}_{l\in[r]_{\mathbb{N}_0}}, 
\end{equation} 

una colección de $r$ simulaciones[<sup>1](#3). En Python podemos hacer una implementación sencilla para generar estos lotes de datos, esto sería algo así:

In [3]:
def generate_batches(m, k, r, sigma):
    """
    Input:
        m: dimension of the space of independent variables.
        size_data: the amount of data to generate by batch.
        batchs: number of batchs.
        sigma: standard deviation.
    
    Output: 
        List with generated batches.        
    """
    
    # We generate the synthetic data for each batch.
    synthetic_data = SyntheticData(m)
    batches = [synthetic_data.generate(k, sigma) for _ in range(r)]
    
    return batches

El objetivo de la ecuación [Ecuación 1](#1) es encontrar un vector $\hat{\theta}\in ERM_{\theta}(S)$ , para nuestro ejercicio esto se traduce en estimar el vector $\hat{\theta}$ tan cerca como sea posible al parámetro conocido $\xi$ del modelo simulado en la [Ecuación 2](#2).

El indicador que se utilizará para comparar la estimación de los coeficientes $\hat{\theta}$ con los coeficientes verdaderos $\xi$ será el error cuadrático medio (RMSE). Esta medición se utilizará para evaluar el rendimiento en cada una de la simulaciones $S_l$. Tenga en cuenta que el RMSE se calculan para un elemento específico en el vector $\theta$. La notación $\operatorname{RMSE}_d$ enfatizará a qué parámetro se refiere. Para los parámetros $d\in [m]_{\mathbb{N_0}}$, considere:

\begin{equation}
\operatorname{RMSE}_d = \left(\frac{1}{r}\sum_{j=0}^{r}\big(\theta^{(d)}_j-\xi^{(d)}\big)^{2}\right)^{\frac{1}{2}}.
\end{equation}

Así se obtiene $\operatorname{RMSE} = (\operatorname{RMSE}_0,\dots, \operatorname{RMSE}_{m})$. En Python se tendría algo así:

In [4]:
def RMSE(matrix_theta: list, xi: list, r: int) -> np.array:
    """
    Input:
        matrix_theta: this a matrix of values theta for each S.
        xi: synthetic parameters.
        r: number of simulations. 
        
    Output:
        RMSE
    """
    
    matrix_theta = np.array(matrix_theta)
    xi = np.array(xi).reshape(-1, 1)
    arg = 1/r * (matrix_theta - xi)**2
    
    return np.sqrt(arg.sum(axis=1))

En el código anterior se espera como parámetro de entrada una matriz $\Theta$ donde cada $l$ - ésima columan es el $l$ - vector de coeficientes aprendido para la $l$ - ésima simulación de datos $X_{l}$, esto se explicará con más detalle a continuación.

Si se asume que $h_{\theta}$ es un predictor para el conjunto de datos $S_{l}=\{X_{l}, y_{l}\}$, entonces el problema de regresión lineal en la [Ecuación 1](#1) se puede reescribir con la siguiente función de perdida:

\begin{equation}\tag{3}
E_{S}(\theta)=\frac{1}{2k}\sum_{i=1}^{k}\big(h_{\theta}(x_i)-y_i\big)^2.
\end{equation}

Esta función para la perdida es facil de implementar en Python como podemos ver a continuación:

In [5]:
def loss(predictor, sample: tuple) -> float:
    """
    Input:
        predictor: linear predictor. 
        sample: dataset.
    
    Output:
        loss.    
    """
    
    arg = list()
    k = len(sample[0])
    
    for i in range(k):
        x_i = sample[0][i]
        y_i = sample[1][i]
        k = len(sample[0])
        arg.append((predictor.predict(x_i) - y_i)**2)
        
    return 1/(2*k) * sum(arg) 

Una forma de resolver este problema es utilizar los métodos del gradiente descente. Estos son algoritmos que hacen uso de las derivadas parciales de funciones direfenciales convexas y, como cualquier algoritmo, su eficiencia puede ser monitoreada. Principalmente, la eficiencia de un algoritmo depende de que tan precisa es la estimación que produce. En segungo lugar, se analiza su capacidad para producir tal estimación dentro de un marco de tiempo factible.

Los métodos del gradiente descendente produce secuencias finitas de números $\theta$. Para ser eficiente esta secuencia debería converger al valor óptimo. En este ejercicio, para algún $\delta > 0$, la eficacia de un algoritmo se evalúa midiendo la duración para producir una estimación $\hat{\theta}$ con precisión $\delta$ tal que:

\begin{equation}
||\hat{\theta}-\xi||<\delta
\end{equation}

En lugar de una respuesta continua $y$, la respuesta podría ser binaria, es decir, $y \in \{0, 1\}$. En estadística, este es un modelo de regresión logística. Dados ciertos predictores, también es un problema de aprendizaje supervisado. Las premisas para resolver un problema de regresión logística son similares a las de un problema continuo. En el caso binario el problema se resuelve para la siguiente función de perdida:

<a id='4'></a>
\begin{equation}\tag{4}
E_{S}(\theta) = \frac{1}{m}\bigg(\sum_{i=1}^{m}\Big(y_i\log h_{\theta}\big(x_i\big) + \big(1-y_i\big)\log \big(1 - h_{\theta}\big(x_i\big)\big)\Big)\bigg)
\end{equation}

donde

\begin{equation}
h_{\theta}(x) = \frac{1}{1+e^{-\theta^\top x}}
\end{equation}

Esta es una función convexa y suave. Por lo tanto, el proceso de solución es similar al de [Ecuación 1](#1). Pero aquí, la variable dependiente $y$ es categórica. Si $y$ tiene más de dos resultados posibles, el modelo se llama regresión logística multinomial.

## Gradiente descendente (GD)

Calcular la solución del objetivo de optimización se logra minimizando la función de costo $E_{S}(\theta)$. Su gradiente se representa por $\nabla E_{S}(\theta)$ y revela en cual direción la función decrese más rápido (<a src ='https://alejandrosanchezyali.blogspot.com/2020/10/algoritmo-del-gradiente-descendente.html'>Ver Gradiente Descendente</a>). Utilizando $\nabla E_{S}(\theta)$ es posible dar pasos repetidamente hacia un mínimo local. Esto sugiere el siguiente algoritmo: 

#### *Algoritmo 1. Gradiente Descendente Clásico*
<blockquote>    
Input: $\theta$, $X$, $Y$, $t$, $ \alpha$, $\nabla E_{S}$
<br>1. &nbsp; &nbsp;&nbsp;for $k=0$ to $p$ do:
<br>2. &nbsp; &nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;$\theta\leftarrow \theta - \alpha \nabla E_{S}(\theta, X, Y)$
<br>3. &nbsp; &nbsp;&nbsp;end    
<br>return: $\theta$

</blockquote>

En el seudocódigo anterior se tiene que $t$ es el número máximo de iteraciones y $\alpha$  es la tasa de aprendizaje, hiperparámetros que son ajustados manualmente. Hay métodos que  permite aproximar la tasa de aprendizaje  para acelerar la convergencia, pero aquí en nuestro caso,  el descenso del gradiente se mantendrá en su forma pura. Cuando $\alpha$ se elige lejos de su valor óptimo $\alpha^*$, el algoritmo divergerá si $\alpha > \alpha^*$ y convergerá muy lentamente si $\alpha < \alpha^*$. 

Recordemos la función objetivo $E(\theta)$ en [Ecuación 1](#1). Sea

\begin{equation}
h_{\theta}(x)= \theta^\top x
\end{equation}

Si el número de ejemplos de entrenamiento es $k$, el número de parámetros es $m+1$, entonces entonces considerando $j=0, \cdots, m$ y $e_0,\dots,e_m$ la base estandar, entonces el gradiente de $E_{S}(\theta)$ se puede expresar como:

\begin{equation}
\nabla E_{S}(\theta) = \sum_{i=0}^{m}\frac{\partial E}{\partial \theta_{i}}e_{i}
\end{equation}

donde

\begin{equation}
\frac{\partial E_{S}}{\partial \theta_{j}}=\frac{1}{k}\sum_{i=1}^{k}\big(\theta^\top x_i-y_i\big)x_{i,j}.
\end{equation}

La convergencia del algoritmo del gradiente descendente depende de la elección precisa de la tasa de aprendizaje $\alpha$, así como del número suficiente de iteraciones $t$. Nuestro caso es un problema de optimización convexo y tomando hiperparámetros apropiados, es decir $0<\alpha <1$, el algoritmo debería converger. 

Para la [Ecuación 4](#4), el objetivo es la optimización para la regresión logística, la función de costo es otra, pero nuevamente podemos obtener el gradiente de la siguiente manera:

\begin{equation}
\frac{\partial E_{S}}{\partial \theta_{j}}=\frac{1}{k}\sum_{i=1}^{k}\big(h_{\theta}(x)-y_i\big)x_{i,j},
\end{equation}

donde $h_{\theta}(x) = \frac{1}{1+e^{-\theta^\top x}}$. Como la ecuación $E_{S}(\theta)$ para la regresión logistica vuelve a ser una función convexa, entonces se puede usar los algoritmos basados en el gradiente para minimizar la [Ecuación 4](#4).

A continuación se presenta la implementación del algoritmo del gradiente descendente en python:

In [6]:
def gd(predictor, X, Y, t, alpha, gradient):
    """
    Input:
        predictor: it's a predictor function.
        X: it's a matrix with the independent data.
        Y: it's a matrix with the dependent data.
        t: it's the maximum number of iterations. 
        alpha: it's the learning rate.
        gradient: it's a gradient function of the predictor function.
    """
    
    theta = predictor.theta
    iterations = 0
    
    while iterations <= t:
        theta = theta - alpha * gradient(theta, X, Y)
        iterations += 1
        
    predictor.theta = theta
    
    return predictor

### Gradiente descendente estocástico (SGD)

Otro algoritmo para resolver problemas de optimización es la versión estocástica del gradiente descendente. En este caso en lugar de evaluar sobretodo el conjunto de entrenamiento $S$, en cada iteración se selecciona aleatoriamente un solo ejemplo $(x_{i_k}, y_{i_k})$, de manera que el gradiente estocástico sería:

\begin{equation}
\nabla E_{S}^{sg}(\theta) = \sum_{i=0}^{m}\frac{\partial E_{S}(\theta, x_{i_k}, y_{i_k})}{\partial \theta_{j}}e_{j},
\end{equation}

en donde:

\begin{equation}
\frac{\partial E_{S}(\theta, x_{i_k}, y_{i_k})}{\partial \theta_{j}} = (h_{\theta}(x_{i_k}) -  y_{i_k})x_{i_k, j}.
\end{equation}

La elección de ${i_k}$ se puede elegir mediante alguna de las siguientes reglas:

1. Elección aleatoria: escoger $i_k\in \{1,2,\dots, m\}$ asumiendo una distribucción uniforme.
2. Elección cíclica: seleccionar ${i_k}=1, 2,\dots, m, 1, 2,\dots, m,\dots$.

La elección aleatoria es la más común en la práctica. 

Por lo tanto, el ciclo se puede inicializar en uno y finalizará cuando se alcance el número máximo de epocas $t$. Este algoritmo tiene la siguiente esquema:

#### *Algoritmo 2. Gradiente Descendente Estocástico*
<blockquote>    
Input: $\theta$, $X$, $Y$, $t$, $ \alpha$, $\nabla E_{S}$, $index\_rule$
<br>1. &nbsp;&nbsp;&nbsp;for $k=1$ to $t$ do:
<br>2. &nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;$i_k = index\_rule(X)$
<br>3. &nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;$\theta\leftarrow \theta - \alpha \nabla E_{S}(\theta, x_{i_k}, y_{i_k})$
<br>4. &nbsp;&nbsp;&nbsp;&nbsp;end    
<br>return: $\theta$
</blockquote>

Observe que el algoritmo 2, $k$ denota el cantidad de datos en el conjunto $S$, consecuentemente, cada epoca recorre todos los elementso de $S$. Este algoritmo tambien se aplica para resolver el problema de la regresión logística. Un implementación rápida de este algoritmo sería:

In [7]:
from itertools import cycle

def sdg(predictor, initial_theta, X, Y, t, alpha, gradient, index_rule):
    theta = predictor.theta
    iterations = 0
    indexes = np.arange(X.shape[0]).tolist()
        
    if index_rule == 'randomized_rule':
        
        while iterations <= t:
            i_k = np.random.choice(indexes)
            theta = theta - alpha * gradient(theta, X[i_k], Y[i_k])
            iterations += 1
            
    elif index_rule == 'cyclic_rule':
        
        pool = cycle(indexes)
        while iterations <= t:
            i_k = next(pool)
            theta = theta - alpha * gradient(theta, X[i_k], Y[i_k])
            iterations += 1
    else:
        raise Exception('A suitable rule has not been defined')
            
        
    predictor.theta = theta
    return predictor

### Gradiente descendente estocástico por minilotes (Mini-Batch SGD)

Otro versión del gradiente descendente y que a menudo logra un mejor rendimiento que SGD y GD es ejecutar el algoritmo por pequeñas partes de los datos. Una parte de los datos se denomina minilote (Mini-batch), mientras que todo el conjunto de entrenamiento $S$ se denomina lote. Estos minilotes se generan seleccionando aleatoriamente los vectores fila de la matriz $X$. El esquema general de este algoritmo sería:

#### *Algoritmo 3. Gradiente Descendente Estocástico por lotes*
<blockquote>    
Input: $\theta$, $X$, $Y$, $t$, $ \alpha$, $\nabla E_{S}$, $size\_batch$
<br>1. &nbsp;&nbsp;&nbsp;for $k=1$ to $t$ do:
<br>2. &nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;$[i_{0, k}, \dots, i_{size\_batch, k}] = choose\_indexes(X)$
<br>3. &nbsp;&nbsp;&nbsp; &nbsp;&nbsp;&nbsp;$\theta\leftarrow \theta - \alpha \nabla E_{S}(\theta, X[i_{0, k}, \dots, i_{size\_batch, k}], Y[i_{0, k}, \dots, i_{size\_batch, k}])$
<br>4. &nbsp;&nbsp;&nbsp;&nbsp;end    
<br>return: $\theta$
</blockquote>

Intuitivamente el algoritmo converge si SGD converge. 

In [8]:
def sdg_mini_batch(initial_theta, X, Y, t, alpha, gradient, size_batch):
    theta = predictor.theta
    iterations = 0
    indexes = np.arange(X.shape[0]).tolist()
        
    while iterations <= t:
        indexes_batch = np.random.choice(indexes, size_batch)
        sgrad = gradient(theta, X[indexes_batch], Y[indexes_batch])
        theta = theta - alpha * sgrad     
        iterations += 1
        
    predictor.theta = theta
    return predictor

### Simulaciones en ambientes controlados para la regresión lineal

A continuación vamos a analizar el comportamiento de los algorimos 1, 2, 3 en un ambiente controlado de regresión lineal. Para esto vamos a generar varios datos de prueba, en los cuales podemos conocer el valor de $\theta$.

#### Análisis con 30 lotes de 200 datos de la forma $(x, y)\in \mathbb{R}^2$  y un ruido del 5%.

In [9]:
m = 3
batches = generate_batches(m=m, k=200, r=10, sigma=.5)

In [10]:
def euclidean_distance(theta, xi):
    arg = theta - xi
    return np.linalg.norm(arg)

Para hacer el análisis de los algoritmos GD, SGD SGD MINI-BATCH, se hace unas pequeñas modificaciones para medir la evolución de la perdida o error en cada una de las iteraciones. 

##### Gradiente descendente clásico moficado:

In [11]:
def gd(predictor, X, Y, t, alpha, gradient, loss=None, xi=None):
    
    theta = predictor.theta
    iterations = 0    
    
    if loss is not None:
        delta_theta = list()
        deltas_thetas = theta.reshape(-1, 1) - xi.reshape(-1, 1)
    
    if loss is not None: 
        errors = list()
        sample = tuple([X, Y])
    
    while iterations <= t:
        
        theta = theta - alpha * gradient(theta, X, Y)
        predictor.theta = theta
        
        if loss is not None:            
            errors.append(loss(predictor=predictor, sample=sample))
            
        if loss is not None:
            delta_theta.append(euclidean_distance(theta, xi))
            arg = theta-xi
            deltas_thetas = np.hstack((deltas_thetas, arg))
            
        iterations += 1   
        
    return predictor, errors, delta_theta, deltas_thetas

##### Gradiente descendente estocástico moficado:

In [12]:
from itertools import cycle

def sgd(predictor, X, Y, t, alpha, gradient, index_rule, loss=None, xi=None):    

    theta = predictor.theta
    iterations = 0
    indexes = np.arange(X.shape[0]).tolist()   
    
    if loss is not None:
        delta_theta = list()
        deltas_thetas = theta.reshape(-1, 1) - xi.reshape(-1, 1)
    
    if loss is not None: 
        errors = list()
        sample = tuple([X, Y])  
        
    if index_rule == 'randomized_rule':
        
        while iterations <= t:
            i_k = np.random.choice(indexes)
            theta = theta - alpha * gradient(theta, X[i_k], Y[i_k])
            predictor.theta = theta
            
            iterations += 1
            
            if loss is not None:            
                errors.append(loss(predictor=predictor, sample=sample))

            if loss is not None:
                delta_theta.append(euclidean_distance(theta, xi))
                arg = theta-xi
                deltas_thetas = np.hstack((deltas_thetas, arg))
            
    elif index_rule == 'cyclic_rule':
        
        pool = cycle(indexes)
        while iterations <= t:
            i_k = next(pool)
            theta = theta - alpha * gradient(theta, X[i_k], Y[i_k])
            predictor.theta = theta
            
            iterations += 1
            
            if loss is not None:            
                errors.append(loss(predictor=predictor, sample=sample))

            if loss is not None:
                delta_theta.append(euclidean_distance(theta, xi))
                arg = theta-xi
                deltas_thetas = np.hstack((deltas_thetas, arg))
                
    else:
        raise Exception('A suitable rule has not been defined')
            
        
    predictor.theta = theta
    return predictor, errors, delta_theta, deltas_thetas

##### Gradiente descendente estocástico minibatch modificado:

In [13]:
def sdg_mini_batch(predictor, X, Y, t, alpha, gradient, size_batch, loss=None, xi=None):
    theta = predictor.theta
    iterations = 0
    indexes = np.arange(X.shape[0]).tolist()
    
    if loss is not None:
        delta_theta = list()
        deltas_thetas = theta.reshape(-1, 1) - xi.reshape(-1, 1)
    
    if loss is not None: 
        errors = list()
        sample = tuple([X, Y])  
        
    while iterations <= t:
        indexes_batch = np.random.choice(indexes, size_batch)
        sgrad = gradient(theta, X[indexes_batch], Y[indexes_batch])
        theta = theta - alpha * sgrad 
        
        if loss is not None:            
            errors.append(loss(predictor=predictor, sample=sample))

        if loss is not None:
            delta_theta.append(euclidean_distance(theta, xi))
            arg = theta-xi
            deltas_thetas = np.hstack((deltas_thetas, arg))
        
        iterations += 1
        
    predictor.theta = theta
    return predictor, errors, delta_theta, deltas_thetas

##### Gradiente para la regresión lineal:

In [14]:
def gradient(theta, X, Y):
    if len(X.shape)==1:
        X = X.reshape(1, -1)
    return X.T.dot((X.dot(theta)- Y))

In [20]:
matrix_theta = list()
errors = list()
deltas_theta = list()
deltas = list()
xi = batches[0][2]
for batch in batches:
    X = batch[0]
    Y = batch[1]

    predictor = LinearPredictor(m)
    parameters = dict(
        predictor=predictor,
        X=X,
        Y=Y,
        t=200,
        alpha=0.001,
        gradient=gradient,
        index_rule='randomized_rule',
        # size_batch=None,
        loss=loss,        
        xi=xi
    )
    predictor, error, delta_theta, delta = sgd(**parameters)
    errors.append(error)
    deltas.append(delta)
    deltas_theta.append(delta_theta)
    matrix_theta.append(predictor.theta)

In [21]:
matrix_theta = np.array(matrix_theta)
matrix_theta = matrix_theta.squeeze().T

In [ ]:
RMSE(matrix_theta=matrix_theta, xi=xi, r=5)

In [ ]:
import matplotlib.pyplot as plt
i = 0
for error in errors:
    plt.plot(error, label='iter {}'.format(i))
    plt.legend()
    plt.title('Loss vs Iterations')
    plt.xlabel('N° Iterations')
    plt.ylabel('Loss Function')
    i += 1

In [ ]:
i = 0
for delta_theta in deltas_theta:
    plt.plot(delta_theta, label='iter {}'.format(i))
    plt.legend()
    plt.title(r'Distance between $\theta$ and $\xi$')
    plt.xlabel('N° Iterations')
    plt.ylabel('Distance')
    i += 1

In [ ]:
fig, axs = plt.subplots(2, 2, figsize=(8, 8))
batches = len(deltas)
switch = True
for j in range(m + 1):
    for k, axis in enumerate(axs.flat):
        for i in range(batches):
            if j == k:
                axis.plot(deltas[i][j])
                axis.set_title(r'$\theta_{}$'.format(j))
plt.show()

In [ ]:
batches

In [ ]:
deltas[0][1]

## Bibliografía
1. Sebastián Ruder. 2017. An overview of gradient descent optimization algorithms. Insight Centre for Data Analytics, Nui Galway.
2. https://mmuratarat.github.io/2019-01-07/logistic-regression-in-Tensorflow

## Notas
<a id="1">[1]</a>. El conjunto $[r]_{\mathbb{N}_0}$ es $\{0, 1, 2,\dots, r\}$.

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

fig = plt.figure(figsize=(15, 15)) 
# create figure window

gs = gridspec.GridSpec(2, 3)
# Creates grid 'gs' of a rows and b columns 


ax = plt.subplot(gs[1, 1])
ax.plot([1, 2], [3, 4])
ax = plt.subplot(gs[1, 2])
ax.plot([1, 2], [3, 4])
fig.add_subplot(ax)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

np.random.seed(19680801)
data = np.random.randn(2, 100)

fig, axs = plt.subplots(2, 3, figsize=(5, 5))
axs[0, 0].hist(data[0])
axs[1, 0].scatter(data[0], data[1])
axs[0, 2].plot(data[0], data[1])
axs[1, 1].hist2d(data[0], data[1])

plt.show()

In [ ]:
6 // 3

In [ ]:
 a = [1]

In [ ]:
deltas[0].shape

In [ ]:
X = np.array([[1,2], [3, 4]])

In [ ]:
len(X.shape)

In [ ]:
Y = np.array([1, 2]).reshape(-1, 1)
Y.shape

In [ ]:
Y.T.dot(Y)